영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.


LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.


예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

In [5]:
import sys
import os
sys.path.append('../..')
from get_env import get_env

# API 키 설정
api_key = get_env()

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import json
from get_env import get_env

api_key = get_env()


In [6]:
# 영화 정보 예제 데이터
movie_examples = [
    {
        "input": "인셉션",
        "output": """영화명: 인셉션 (Inception)
감독: 크리스토퍼 놀란
주요 출연진: 레오나르도 디카프리오, 마리옹 코티야르, 톰 하디, 엘리엇 페이지, 조셉 고든-레빗
예산: 약 1억 6천만 달러
흥행 수익: 약 8억 3천만 달러
장르: SF, 액션, 스릴러
시놉시스: 꿈 속에서 다른 사람의 생각을 훔치는 '인셉션' 기술을 다루는 영화로, 도미노 코브(레오나르도 디카프리오)가 거대한 기업의 후계자에게 특별한 임무를 수행하는 이야기입니다."""
    },
    {
        "input": "어벤져스: 엔드게임",
        "output": """영화명: 어벤져스: 엔드게임 (Avengers: Endgame)
감독: 앤서니 루소, 조 루소
주요 출연진: 로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요한슨
예산: 약 3억 5천만 달러
흥행 수익: 약 27억 9천만 달러
장르: 액션, 어드벤처, SF
시놉시스: 타노스의 스냅으로 절반의 생명체가 사라진 후, 남은 어벤져스들이 시간 여행을 통해 과거로 돌아가 타노스를 막고 사라진 동료들을 되살리려는 이야기입니다."""
    },
    {
        "input": "기생충",
        "output": """영화명: 기생충 (Parasite)
감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정, 최우식, 박소담
예산: 약 1,100만 달러
흥행 수익: 약 2억 5천만 달러
장르: 드라마, 스릴러, 코미디
시놉시스: 반지하 아파트에 사는 기택 가족이 IT 기업 CEO 박사장의 집에 취업하면서 벌어지는 계급 갈등과 사회적 비극을 그린 영화입니다."""
    }
]


In [7]:
# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="영화명: {input}\n{output}"
)

# FewShotPromptTemplate 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=movie_examples,
    example_prompt=example_prompt,
    prefix="다음은 영화에 대한 정보를 제공하는 예시입니다. 주어진 영화명에 대해 동일한 형식으로 정보를 제공해주세요.",
    suffix="영화명: {movie_name}\n",
    input_variables=["movie_name"]
)

print("FewShotPromptTemplate 예시:")
print(few_shot_prompt.format(movie_name="타이타닉"))


FewShotPromptTemplate 예시:
다음은 영화에 대한 정보를 제공하는 예시입니다. 주어진 영화명에 대해 동일한 형식으로 정보를 제공해주세요.

영화명: 인셉션
영화명: 인셉션 (Inception)
감독: 크리스토퍼 놀란
주요 출연진: 레오나르도 디카프리오, 마리옹 코티야르, 톰 하디, 엘리엇 페이지, 조셉 고든-레빗
예산: 약 1억 6천만 달러
흥행 수익: 약 8억 3천만 달러
장르: SF, 액션, 스릴러
시놉시스: 꿈 속에서 다른 사람의 생각을 훔치는 '인셉션' 기술을 다루는 영화로, 도미노 코브(레오나르도 디카프리오)가 거대한 기업의 후계자에게 특별한 임무를 수행하는 이야기입니다.

영화명: 어벤져스: 엔드게임
영화명: 어벤져스: 엔드게임 (Avengers: Endgame)
감독: 앤서니 루소, 조 루소
주요 출연진: 로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요한슨
예산: 약 3억 5천만 달러
흥행 수익: 약 27억 9천만 달러
장르: 액션, 어드벤처, SF
시놉시스: 타노스의 스냅으로 절반의 생명체가 사라진 후, 남은 어벤져스들이 시간 여행을 통해 과거로 돌아가 타노스를 막고 사라진 동료들을 되살리려는 이야기입니다.

영화명: 기생충
영화명: 기생충 (Parasite)
감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정, 최우식, 박소담
예산: 약 1,100만 달러
흥행 수익: 약 2억 5천만 달러
장르: 드라마, 스릴러, 코미디
시놉시스: 반지하 아파트에 사는 기택 가족이 IT 기업 CEO 박사장의 집에 취업하면서 벌어지는 계급 갈등과 사회적 비극을 그린 영화입니다.

영화명: 타이타닉



In [8]:
# LLM 설정
chat = ChatOpenAI(
    temperature=0.7,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 영화 정보 체인 생성 (pipe 연산자 사용)
movie_chain = few_shot_prompt | chat

def get_movie_info(movie_name):
    """영화 정보를 가져오는 함수"""
    try:
        result = movie_chain.invoke({"movie_name": movie_name})
        return result.content
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

# 테스트
print("=== 영화 정보 체인 테스트 ===")
test_movies = ["타이타닉", "어벤져스", "겨울연가"]

for movie in test_movies:
    print(f"\n{'='*50}")
    print(f"영화: {movie}")
    print('='*50)
    result = get_movie_info(movie)
    print(result)


=== 영화 정보 체인 테스트 ===

영화: 타이타닉
영화명: 타이타닉 (Titanic)
감독: 제임스 카메론
주요 출연진: 레오나르도 디카프리오, 케이트 윈슬렛, 빌리 제인, 케이시 블라이스
예산: 약 2억 0천만 달러
흥행 수익: 약 19억 6천만 달러
장르: 로맨스, 드라마
시놉시스: 타이타닉호 침몰 사건을 배경으로 한 제작자와 승객 간의 사랑 이야기를 그린 영화로, 레오나르도 디카프리오와 케이트 윈슬렛이 주연을 맡았습니다.영화명: 타이타닉 (Titanic)
감독: 제임스 카메론
주요 출연진: 레오나르도 디카프리오, 케이트 윈슬렛, 빌리 제인, 케이시 블라이스
예산: 약 2억 0천만 달러
흥행 수익: 약 19억 6천만 달러
장르: 로맨스, 드라마
시놉시스: 타이타닉호 침몰 사건을 배경으로 한 제작자와 승객 간의 사랑 이야기를 그린 영화로, 레오나르도 디카프리오와 케이트 윈슬렛이 주연을 맡았습니다.

영화: 어벤져스
영화명: 어벤져스 (The Avengers)
감독: 조스 웨던
주요 출연진: 로버트 다우니 주니어, 크리스 헴스워스, 크리스 에반스, 스칼렛 요한슨, 마크 러팔로
예산: 약 2억 달러
흥행 수익: 약 15억 4천만 달러
장르: 액션, SF
시놉시스: 지구를 위협하는 위기에 맞서 모여 히어로들이 결집하여 함께 싸우는 어벤져스의 이야기를 다룬 영화입니다.영화명: 어벤져스 (The Avengers)
감독: 조스 웨던
주요 출연진: 로버트 다우니 주니어, 크리스 헴스워스, 크리스 에반스, 스칼렛 요한슨, 마크 러팔로
예산: 약 2억 달러
흥행 수익: 약 15억 4천만 달러
장르: 액션, SF
시놉시스: 지구를 위협하는 위기에 맞서 모여 히어로들이 결집하여 함께 싸우는 어벤져스의 이야기를 다룬 영화입니다.

영화: 겨울연가
영화명: 겨울연가 (A Winter's Tale)
감독: 에릭 슈미트
주요 출연진: 콜린 퍼스, 제니퍼 코넬리, 러셀 크로우, 제시카 브라운 파인드리
예산: 약 4천만 달러
흥행 수익: 약 3천만 달러
장르: 로맨스, 판타지, 드라

In [12]:
# 대화형 영화 정보 검색 시스템
class MovieInfoSystem:
    def __init__(self):
        self.chain = movie_chain
    
    def search_movie(self, movie_name):
        """영화 정보 검색"""
        print(f"'{movie_name}' 영화 정보를 검색 중...")
        result = self.chain.invoke({"movie_name": movie_name})
        return result.content
    
    def interactive_search(self):
        """대화형 검색 모드"""
        print("🎬 영화 정보 검색 시스템")
        print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
        print("-" * 50)
        
        while True:
            movie_name = input("\n영화명을 입력하세요: ").strip()
            
            if movie_name.lower() in ['quit', 'exit', '종료']:
                print("검색을 종료합니다.")
                break
            
            if not movie_name:
                print("영화명을 입력해주세요.")
                continue
            
            try:
                result = self.search_movie(movie_name)
                print(f"\n{result}")
            except Exception as e:
                print(f"오류가 발생했습니다: {e}")

# 시스템 인스턴스 생성
movie_system = MovieInfoSystem()

# 대화형 검색 실행 (주석 해제하여 사용)
# movie_system.interactive_search()


In [13]:
# 간단한 사용 예시
print("=== 영화 정보 체인 사용 예시 ===")

# 개별 영화 검색
print("\n1. 개별 영화 검색:")
movie_info = get_movie_info("해리포터")
print(movie_info)

print("\n2. 시스템을 통한 검색:")
system_result = movie_system.search_movie("겨울연가")
print(system_result)


=== 영화 정보 체인 사용 예시 ===

1. 개별 영화 검색:
영화명: 해리포터 (Harry Potter)
감독: 크리스콜럼버스, 알폰소 쿠아론, 데이빗 예이츠
주요 출연진: 다니엘 래드클리프, 엠마 왓슨, 루퍼트 그린트, 앨런 릭먼, 매기 스미스
예산: 약 1억 2천만 달러
흥행 수익: 약 7억 9천만 달러
장르: 판타지, 모험
시놉시스: 마법 세계를 배우고 성장해나가는 소년 해리 포터의 모험을 다룬 시리즈로, 호그와트 마법학교에서의 이야기와 악당 볼드모트와의 전투가 주된 줄거리입니다.영화명: 해리포터 (Harry Potter)
감독: 크리스콜럼버스, 알폰소 쿠아론, 데이빗 예이츠
주요 출연진: 다니엘 래드클리프, 엠마 왓슨, 루퍼트 그린트, 앨런 릭먼, 매기 스미스
예산: 약 1억 2천만 달러
흥행 수익: 약 7억 9천만 달러
장르: 판타지, 모험
시놉시스: 마법 세계를 배우고 성장해나가는 소년 해리 포터의 모험을 다룬 시리즈로, 호그와트 마법학교에서의 이야기와 악당 볼드모트와의 전투가 주된 줄거리입니다.

2. 시스템을 통한 검색:
'겨울연가' 영화 정보를 검색 중...
영화명: 겨울연가 (Winter Sonata)
감독: 여진왕
주요 출연진: 배용준, 최지우, 박은혜, 박해진
장르: 멜로로맨스, 드라마
시놉시스: 한국 드라마의 대표작으로 송중기 역으로 활약한 주인공과 그에게 첫사랑을 전하는 여주인공의 이야기를 그린 작품입니다.영화명: 겨울연가 (Winter Sonata)
감독: 여진왕
주요 출연진: 배용준, 최지우, 박은혜, 박해진
장르: 멜로로맨스, 드라마
시놉시스: 한국 드라마의 대표작으로 송중기 역으로 활약한 주인공과 그에게 첫사랑을 전하는 여주인공의 이야기를 그린 작품입니다.
